# Webscraping
## Portal da Queixa
### Changes may be required due to Portal da Queixa's continous updates

(c) Nickolas Lago 2021 - Rev. 1.0

### Load packages and do the initializations

In [36]:
# Load libraries
import numpy as np
import pandas as pd
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
import category_encoders as ce
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from spacy import displacy
from collections import Counter

### Importing the data from the provious webscraping

Delete the Unnamed: 0 column and make the complainID the index column of the DataFrame

In [37]:
df = pd.read_excel("complains_data.xlsx", index_col="complainID")

In [38]:
cols_to_drop = ["Unnamed: 0"]
df = df.drop(cols_to_drop, axis=1)

In [39]:
df.head()

,complainStatus,complainUser,complainTitle,complainViews,complainText,complainDate
complainID,,,,,,
59476521,Aguarda resposta,Susana,Lefties - Devolução de artigos,55,Recebi a minha encomenda no dia 19-04-2021 e i...,5 de maio 2021
58935721,Aguarda resposta,Cassia Barcelos,Lefties - Troca de peças compradas on-line,175,"Boa noite, _x000D_\nVenho mostrar o meu total ...",23 de abril 2021
58754421,Aguarda resposta,Milene,Lefties - Encomenda não entregue,62,A minha encomenda que fiz no dia 19 de Março n...,20 de abril 2021
58696921,Aguarda resposta,Mariana Ferreira,Lefties - Encomenda não entregue nem resolvem ...,41,Encomenda nr ‪90003989775‬ já deveria ter sido...,19 de abril 2021
58650521,Resolvida,Olga Santos,Lefties - Encomenda não entregue,124,Bom dia. Fiz uma encomenda online na Lefties n...,17 de abril 2021


## Working with Text

Cleaning and normalization of the raw text.

In [40]:
def textPreProcess(rawText, lowerText=True, charsToRemove=r'\_x000d_|\?|\.|\!|\;|\.|\"|\,|\(|\)|\&|\:|\-|\/', removeNumbers=True, removeLineBreaks=True, removeStopWords=True):
    cleanedText = []
    for x in rawText[:]:
        procText = x
        if lowerText:
            procText = procText.lower()
        if len(charsToRemove)>0:
            procText = re.sub(charsToRemove,' ',procText)
        if removeNumbers:
            procText = re.sub(r'\d+',' ',procText)
        if removeLineBreaks:
            procText = procText.replace('\n',' ').replace('\r', '')
        if removeStopWords:
            stop_words = set(stopwords.words('portuguese'))
            tokenizedText = word_tokenize(procText, language='portuguese')
            cText = []
            for t in tokenizedText:
                if t not in stop_words:
                    cText.append(t)
            procText = ""
            for w in cText:
                procText = procText + w + ' '
            procText = procText.rstrip() # rstring (remove spaces at the right of the string)

        if procText != "":
            cleanedText.append(procText)
    return cleanedText

### Adding a column using the function to the original data frame with the text processing

In [41]:
df["processText"] = pd.DataFrame(data=textPreProcess(df.complainText, removeNumbers=False), index=df.index)

In [42]:
df

,complainStatus,complainUser,complainTitle,complainViews,complainText,complainDate,processText
complainID,,,,,,,
59476521,Aguarda resposta,Susana,Lefties - Devolução de artigos,55,Recebi a minha encomenda no dia 19-04-2021 e i...,5 de maio 2021,recebi encomenda dia 19 04 2021 imediatamente ...
58935721,Aguarda resposta,Cassia Barcelos,Lefties - Troca de peças compradas on-line,175,"Boa noite, _x000D_\nVenho mostrar o meu total ...",23 de abril 2021,boa noite venho mostrar total descontentamento...
58754421,Aguarda resposta,Milene,Lefties - Encomenda não entregue,62,A minha encomenda que fiz no dia 19 de Março n...,20 de abril 2021,encomenda fiz dia 19 março entregue devido ctt...
58696921,Aguarda resposta,Mariana Ferreira,Lefties - Encomenda não entregue nem resolvem ...,41,Encomenda nr ‪90003989775‬ já deveria ter sido...,19 de abril 2021,encomenda nr ‪90003989775‬ deveria ter sido en...
58650521,Resolvida,Olga Santos,Lefties - Encomenda não entregue,124,Bom dia. Fiz uma encomenda online na Lefties n...,17 de abril 2021,bom dia fiz encomenda online lefties dia 1 abr...
...,...,...,...,...,...,...,...
8776417,Sem resolução,Maria Fernandes,Lefties - Hematoma grave na face,1305,Os sapatos prateados estavam na prateleira aci...,13 de março 2017,sapatos prateados prateleira acima altura pres...
7865917,Sem resolução,patricia vidal,Lefties - Mau atendimento,1829,Bom dia.Ja por diversas vezes me dirigi a voss...,23 de janeiro 2017,bom dia ja diversas vezes dirigi vossa loja aq...
5810316,Sem resolução,Julia Almeida,Lefties - Troca,3777,Boa tarde fui a lefties C.C. Arrábida fiz uma ...,19 de setembro 2016,boa tarde lefties c c arrábida fiz compra valo...


## Other Processing Functions

In [43]:
# Tokenize texts
def tokenize_words(texts):
    words_new = []
    for w in (texts[:]):
        w_token = word_tokenize(w)
        if w_token != '':
            words_new.append(w_token)
    return words_new

In [44]:
# Stemm texts
def stemming(words):
    procText = []
    for w in (words[:]):
        stemmed_word = [snowball.stem(x) for x in (w[:])]
        procText.append(stemmed_word)
    return procText

# Analysis

In [45]:
df["processText"]

complainID
59476521    recebi encomenda dia 19 04 2021 imediatamente ...
58935721    boa noite venho mostrar total descontentamento...
58754421    encomenda fiz dia 19 março entregue devido ctt...
58696921    encomenda nr ‪90003989775‬ deveria ter sido en...
58650521    bom dia fiz encomenda online lefties dia 1 abr...
                                  ...                        
8776417     sapatos prateados prateleira acima altura pres...
7865917     bom dia ja diversas vezes dirigi vossa loja aq...
5810316     boa tarde lefties c c arrábida fiz compra valo...
5716216     comprei umas sapatilhas promoção disseram mês ...
3071916     altura natal filhas receberam prendas brinqued...
Name: processText, Length: 177, dtype: object

In [46]:
# Create a dataframe with only the description

complainsData = pd.DataFrame(data=df, index=df.index, columns=['complainText', 'processText'])

In [47]:
# Tokenize text
complainsData['Words'] =  tokenize_words(complainsData['processText'])

In [48]:
# Load Spacy English model
nlp = spacy.load("pt_core_news_sm")

In [49]:
# Check entities in review 
print(df['processText'][58935721])
doc = nlp(df['processText'][58935721])
print([(X.text, X.label_) for X in doc.ents])

boa noite venho mostrar total descontentamento frustação relativamente serviço prestado fiz encomenda várias peças pack 4 calças fato treino 15 99€ chegar encomenda deparei vez pack recebido 1 calça vez 4 enviei mail apoio cliente expor situação passado algumas horas responderam afirmar entrado contacto comigo várias vezes sucesso mentira pois confirmei número telefone cedido conta nenhuma chamada atendida pediram retribuir chamada contactei então chamada telefonica assistente atendeu nunca pediu desculpa sucedido explicou deveria devolver calça etiqueta própria enviado pack completo passado dois dias recebi pack desta vez 1 4 calças veio vários tamanhos abaixo pedido quero resolvam situação vez todas trabalho compatível ponto picket fazer devolução espero resolvam desagradável situação vez todas concertesa vou recomendar comprar nada marca
[]


In [50]:
# Check entities in review 
print(df['processText'][56382721])
doc = nlp(df['processText'][56382721])
print([(X.text, X.label_) for X in doc.ents])

fiz encomenda dia 10 fevereiro hoje chegou nada mandaram mail dizer conseguiam entregar erro morada 4° encomenda outras 3 qualquer problema têm contacto caso conseguirem chegar morada nunca contactaram
[]


In [51]:
# Count the labels
labels = [x.label_ for x in doc.ents]
Counter(labels)

Counter()

In [52]:
# Entities visualization
displacy.render(doc, jupyter=True, style='ent')

/Users/lagonick/opt/anaconda3/lib/python3.7/site-packages/spacy/displacy/__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [55]:
# For example, if our objective was understand what guests say about the staff language skills we could look for reviews that mention languages
counter=0   # to stop after x for demostration speed
annReviews=[]
for r in df['processText']:
  doc = nlp(r)
  for i in doc.ents:
      if i.label_=='DATE':
          annReviews.append(r)
          counter = counter + 1
          break
  if counter>=3:    # Stop after the first three reviews have been found
      break

annReviews

[]

In [61]:
complainsData['processText']

complainID
59476521    recebi encomenda dia 19 04 2021 imediatamente ...
58935721    boa noite venho mostrar total descontentamento...
58754421    encomenda fiz dia 19 março entregue devido ctt...
58696921    encomenda nr ‪90003989775‬ deveria ter sido en...
58650521    bom dia fiz encomenda online lefties dia 1 abr...
                                  ...                        
8776417     sapatos prateados prateleira acima altura pres...
7865917     bom dia ja diversas vezes dirigi vossa loja aq...
5810316     boa tarde lefties c c arrábida fiz compra valo...
5716216     comprei umas sapatilhas promoção disseram mês ...
3071916     altura natal filhas receberam prendas brinqued...
Name: processText, Length: 177, dtype: object